In [3]:
import sys
sys.path.append("..")
import comet_ml
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import torch

from neuro_ix.datasets.mriqc_dataset import MIDLMRArtDataset
api = comet_ml.API(
            api_key="WmA69YL7Rj2AfKqwILBjhJM3k"
        )

## Get Dataset

In [ ]:
ds = MIDLMRArtDataset